**Домашнее задание №1**

1. Создайте Яндекс-акааунт
2. Зарегестрируйтесь на Yandex Cloud (создайте платежный аккаунт)
3. Создайте сервисный аккаунт
4. Изучите возможности сервиса [СИНТЕЗ РЕЧИ](https://cloud.yandex.ru/docs/speechkit/tts/?from=int-console-empty-state)
5. Практическое задание:
  - запишите с помощью микрофона произвольное аудио (длительностью 15-20 секунд)
  - распознайте записанное аудио с помощью сервисов распознавания речи
  - выполните синтез распознанного текста и выведите результат в ячейку колаба

**Практика**

**1. запишите с помощью микрофона произвольное аудио (длительностью 15-20 секунд)**

In [1]:
!pip -q install ffmpeg-python
!pip -q install pydub

In [2]:
import gdown 
gdown.download('https://storage.yandexcloud.net/aiueducation/Intensive/v2.0/micro.py', None, quiet=True)
import micro
import scipy

In [25]:
def record_audio():
    # Запустим процесс записи с микрофона
    audio, sr = micro.get_audio()

    # Сохраним запись в файл recording.wav
    scipy.io.wavfile.write('recording.wav', sr, audio)

    # Преобразуем в формат ogg
    !rm -rf result.ogg
    !ffmpeg -i recording.wav -c:a libvorbis -q:a 4 recording.ogg > /dev/null 2>&1

    print('Аудио сохранено в файл: "recording.ogg"')

In [26]:
record_audio()

Аудио сохранено в файл: "recording.ogg"


**2. распознайте записанное аудио с помощью сервисов распознавания речи**

In [8]:
from getpass import getpass
# получим все секреты

OAuth_token = getpass()
API_KEY = getpass()
ID_FOLDER = getpass() 
 

··········


In [16]:
!curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 54.9M  100 54.9M    0     0  95.6M      0 --:--:-- --:--:-- --:--:-- 95.4M


In [17]:
!unzip -q awscliv2.zip

In [18]:
!sudo ./aws/install

You can now run: /usr/local/bin/aws --version


In [12]:
import requests
import json
import time
import shutil 
import wave
from pathlib import Path

def yandex_recognize_api(fname, API_KEY, LANGUAGE_CODE='ru-RU'):
    !rm -rf tmp_folder
    !mkdir tmp_folder

    fname_path =  Path(fname)
    
    # путь к файлу может содержать и папку
    fname_only = fname_path.name
    
    shutil.copy(fname, f"tmp_folder/{fname_only}")

    # Расчитываем значение sampleRate файла
    with wave.open(fname, "rb") as wave_file:
      frame_rate = wave_file.getframerate()
    # Загружаем файл в бакет
    !aws --endpoint-url=https://storage.yandexcloud.net s3 cp --recursive tmp_folder/ s3://terra-bucket-dm

    # URL-адрес метода longRunningRecognize
    URL = 'https://transcribe.api.cloud.yandex.net/speech/stt/v2/longRunningRecognize'
    time.sleep(2)
    # Определение тела запроса
    request_body = {
        'config': {
            'specification': {
                'languageCode': LANGUAGE_CODE,
                'audioEncoding': 'LINEAR16_PCM',
                'sampleRateHertz': frame_rate,
            }
        },
       "audio": {
        "uri": f"https://storage.yandexcloud.net/terra-bucket-dm/{fname_only}"
      }
    }
    print("Отправка запроса на сервер Yandex SpeechKit")
    # Отправка запроса на сервер Yandex SpeechKit
    response = requests.post(
        URL,
        headers={'Authorization': f'Api-Key {API_KEY}'},
        json=request_body
    )
    if response.status_code == 200:
        print("Получение ID задачи распознавания")

        # Получение ID задачи распознавания
        operation_id = response.json()['id']
        
        # Проверка статуса задачи распознавания и получение результата
        while True:
            response = requests.get(
                f'https://operation.api.cloud.yandex.net/operations/{operation_id}',
                headers={'Authorization': f'Api-Key {API_KEY}'}
            )

            if response.json()['done']:
                # Распознанный текст
                recognized_text = response.json()['response']['chunks'][0]['alternatives'][0]['text']
                return recognized_text
            else:
                time.sleep(1)
    else:
        print('Ошибка распознавания речи. Код ошибки: {}'.format(response.status_code))
        print(response.text)

In [27]:
recognized_text = yandex_recognize_api(
    fname = '/content/recording.wav',
    API_KEY = API_KEY,
)

upload: tmp_folder/recording.wav to s3://terra-bucket-dm/recording.wav
Отправка запроса на сервер Yandex SpeechKit
Получение ID задачи распознавания


In [28]:
print(recognized_text)

1 2 3 4 5 вышел зайчик погулять


**3. выполните синтез распознанного текста и выведите результат в ячейку колаба**

In [23]:
import argparse
import requests

def synthesize(folder_id, API_KEY, text):
   url = 'https://tts.api.cloud.yandex.net/speech/v1/tts:synthesize'
   
   headers={'Authorization': f'Api-Key {API_KEY}'}

   data = {
       'text': text,
       'lang': 'ru-RU',
       'voice': 'filipp',
       'folderId': folder_id
   }

   with requests.post(url, headers=headers, data=data, stream=True) as resp:
       if resp.status_code != 200:
           raise RuntimeError("Invalid response received: code: %d, message: %s" % (resp.status_code, resp.text))

       for chunk in resp.iter_content(chunk_size=None):
           yield chunk

In [32]:
from IPython.display import Audio, display

audio_file = "synthesize.wav"

try:
  with open(audio_file, "wb") as f:
         for audio_content in synthesize(ID_FOLDER, API_KEY, recognized_text):
             f.write(audio_content)
  print(f"Файл {audio_file} готов")
  # воспроизводим
  display(Audio(audio_file, autoplay=True))
except Exception as ex:
  print(f"Ошибка: {ex}")

Файл synthesize.wav готов
